In [ ]:
!pip install numpy==1.26.4 rdkit-pypi pubchempy

In [ ]:
import pubchempy as pcp
from rdkit import Chem
from rdkit.Chem import Draw

In [ ]:
compound = pcp.get_compounds("(s)-ibuprofen", "name")[0]
smiles = compound.isomeric_smiles
print(smiles)

In [ ]:
# Convert to RDKit molecule and draw
mol = Chem.MolFromSmiles(smiles)
Draw.MolToImage(mol)

In [ ]:
from rdkit.Chem import Descriptors

print("Molecular Weight:", Descriptors.MolWt(mol))
print("logP (Octanol/Water):", Descriptors.MolLogP(mol))
print("Number of H-bond donors:", Descriptors.NumHDonors(mol))
print("Number of H-bond acceptors:", Descriptors.NumHAcceptors(mol))
print("TPSA (Polar Surface Area):", Descriptors.TPSA(mol))
print("Rotatable Bonds:", Descriptors.NumRotatableBonds(mol))
print("Num Aromatic Rings:", Descriptors.NumAromaticRings(mol))


In [ ]:
from rdkit import DataStructs
from rdkit.Chem import AllChem

# Define some example molecules
names = ["ibuprofen", "aspirin", "paracetamol", "caffeine", "naproxen", "cyclosporine"]
mols = [Chem.MolFromSmiles(pcp.get_compounds(name, "name")[0].isomeric_smiles) for name in names]

# Generate Morgan fingerprints (ECFP4)
fps = [AllChem.GetMorganFingerprintAsBitVect(m, radius=2, nBits=2048) for m in mols]

# Similarity to ibuprofen (first entry)
similarities = [DataStructs.TanimotoSimilarity(fps[0], fp) for fp in fps]

# Show results
for name, sim in zip(names, similarities):
    print(f"{name.capitalize():<12}: similarity = {sim:.3f}")


In [ ]:
from rdkit.Chem import Draw

# Add similarity scores as legends
legends = [f"{name.capitalize()}\nSimilarity: {sim:.2f}" for name, sim in zip(names, similarities)]

# Display molecules in grid
Draw.MolsToGridImage(mols, molsPerRow=3, subImgSize=(250,250), legends=legends)


In [ ]:
def passes_lipinski(mol):
    return (Descriptors.MolLogP(mol) < 5 and
            Descriptors.MolWt(mol) < 500 and
            Descriptors.NumHDonors(mol) <= 5 and
            Descriptors.NumHAcceptors(mol) <= 10)

for name, m in zip(names, mols):
    print(f"{name:<12} passes Lipinski? {passes_lipinski(m)}")


Cyclosporine is an immunosupressant medication and does not follow the rule of 5